<center><h1>第七章 缺失数据</h1></center>

## 一、缺失值的统计和删除
### 1. 缺失信息的统计

缺失数据可以使用`isna`或`isnull`（两个函数没有区别）来查看每个单元格是否缺失，结合`mean`可以计算出每列缺失值的比例

- isnan/isnull

In [2]:
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv('C:\datawhale\joyful-pandas-master\data\learn_pandas.csv', usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight', 'Transfer'])
df.isna().head()#返回bool值，只要是缺测值的位置，返回true

Grade   Name  Gender  Height  Weight  Transfer
0  False  False   False   False   False     False
1  False  False   False   False   False     False
2  False  False   False   False   False     False
3  False  False   False    True   False     False
4  False  False   False   False   False     False

In [13]:
df.isna().mean() # 查看每列true的比例

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

In [14]:
df.isna().mean(1).head()# 查看每行true的比例

0    0.000000
1    0.000000
2    0.000000
3    0.166667
4    0.000000
dtype: float64

如果想要查看某一列缺失或者非缺失的行，可以利用`Series`上的`isna`或者`notna`进行布尔索引。例如，查看身高缺失的行：

In [4]:
df[df.Height.isna()].head()

Grade          Name  Gender  Height  Weight Transfer
3   Sophomore  Xiaojuan Sun  Female     NaN    41.0        N
12     Senior      Peng You  Female     NaN    48.0      NaN
26     Junior     Yanli You  Female     NaN    48.0        N
36   Freshman  Xiaojuan Qin    Male     NaN    79.0        Y
60   Freshman    Yanpeng Lv    Male     NaN    65.0        N

如果想要同时对几个列，检索出全部为缺失或者至少有一个缺失或者没有缺失的行，可以使用`isna, notna`和`any, all`的组合。例如，对身高、体重和转系情况这3列分别进行这三种情况的检索：

- notna

In [17]:
sub_set = df[['Height', 'Weight', 'Transfer']]
df[sub_set.isna().all(1)] # subset提取出需要判断的列，这里all里面是1，代表查看行是否全部缺失

Grade          Name Gender  Height  Weight Transfer
102  Junior  Chengli Zhao   Male     NaN     NaN      NaN

In [19]:
df[sub_set.isna().any(1)].head(3) # 查看至少有一个缺失的行

Grade          Name  Gender  Height  Weight Transfer
3   Sophomore  Xiaojuan Sun  Female     NaN    41.0        N
9      Junior       Juan Xu  Female   164.8     NaN        N
12     Senior      Peng You  Female     NaN    48.0      NaN

In [22]:
sub_set.notna().head()#notna是在有缺测的时候返回false，与isna相反

Height  Weight  Transfer
0    True    True      True
1    True    True      True
2    True    True      True
3   False    True      True
4    True    True      True

In [23]:
df[sub_set.notna().all(1)].head() # 没有缺失

Grade            Name  Gender  Height  Weight Transfer
0   Freshman    Gaopeng Yang  Female   158.9    46.0        N
1   Freshman  Changqiang You    Male   166.5    70.0        N
2     Senior         Mei Sun    Male   188.9    89.0        N
4  Sophomore     Gaojuan You    Male   174.0    74.0        N
5   Freshman     Xiaoli Qian  Female   158.0    51.0        N

### 2. 缺失信息的删除

数据处理中经常需要根据缺失值的大小、比例或其他特征来进行行样本或列特征的删除，`pandas`中提供了`dropna`函数来进行操作。

`dropna`的主要参数为轴方向`axis`（默认为0，即删除行）、删除方式`how`、删除的非缺失值个数阈值`thresh`（$\color{red}{非缺失值}$没有达到这个数量的相应维度会被删除）、备选的删除子集`subset`，其中`how`主要有`any`和`all`两种参数可以选择。

例如，删除身高体重至少有一个缺失的行：

In [8]:
res = df.dropna(how = 'any', subset = ['Height', 'Weight'])
res.shape

(174, 6)

例如，删除超过15个缺失值的列：

In [9]:
res = df.dropna(1, thresh=df.shape[0]-15) # 身高被删除
res.head()

Grade            Name  Gender  Weight Transfer
0   Freshman    Gaopeng Yang  Female    46.0        N
1   Freshman  Changqiang You    Male    70.0        N
2     Senior         Mei Sun    Male    89.0        N
3  Sophomore    Xiaojuan Sun  Female    41.0        N
4  Sophomore     Gaojuan You    Male    74.0        N

当然，不用`dropna`同样是可行的，例如上述的两个操作，也可以使用布尔索引来完成：

In [10]:
res = df.loc[df[['Height', 'Weight']].notna().all(1)]
res.shape

(174, 6)

In [11]:
res = df.loc[:, ~(df.isna().sum()>15)]
res.head()

Grade            Name  Gender  Weight Transfer
0   Freshman    Gaopeng Yang  Female    46.0        N
1   Freshman  Changqiang You    Male    70.0        N
2     Senior         Mei Sun    Male    89.0        N
3  Sophomore    Xiaojuan Sun  Female    41.0        N
4  Sophomore     Gaojuan You    Male    74.0        N

## 二、缺失值的填充和插值
### 1. 利用fillna进行填充

在`fillna`中有三个参数是常用的：`value, method, limit`。其中，`value`为填充值，可以是标量，也可以是索引到元素的字典映射；`method`为填充方法，有用前面的元素填充`ffill`和用后面的元素填充`bfill`两种类型，`limit`参数表示连续缺失值的最大填充次数。

下面构造一个简单的`Series`来说明用法：

- fillna

In [24]:
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan], list('aaabcd'))#索引为abc
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [25]:
s.fillna(method='ffill') # 用前面的值向后填充

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [31]:
s.fillna(method='ffill', limit=1) # 连续出现的缺失，最多填充一次

a    NaN
a    1.0
a    1.0
b    NaN
c    2.0
d    2.0
dtype: float64

In [28]:
s.fillna(s.mean()) # value为标量，用平均值填充所有缺测的地方

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [32]:
s.fillna({'a': 100, 'd': 200}) # 通过索引映射填充的值，建立字典映射

a    100.0
a      1.0
a    100.0
b      NaN
c      2.0
d    200.0
dtype: float64

In [34]:
df.groupby('Grade')['Height'].transform(lambda x: x.fillna(x.mean())).head()#先分组，再将分组的每一组数据传入transform应用函数
#将缺测值用传入的平均值填充

0    158.900000
1    166.500000
2    188.900000
3    163.075862
4    174.000000
Name: Height, dtype: float64

#### 【练一练】
对一个序列以如下规则填充缺失值：如果单独出现的缺失值，就用前后均值填充，如果连续出现的缺失值就不填充，即序列`[1, NaN, 3, NaN, NaN]`填充后为`[1, 2, 3, NaN, NaN]`，请利用`fillna`函数实现。（提示：利用`limit``参数）

In [35]:
s = pd.Series([np.nan, 1, np.nan, 2,np.nan, np.nan])

In [42]:
a=s.fillna(method='ffill', limit=1)

In [41]:
b=s.fillna(method='bfill', limit=1)

In [58]:
condition=s.isna()&a.notna() & b.notna()

In [77]:
np.where(condition==True)[0][0]#单独出现缺测值的位置

2

In [79]:
s[np.where(condition==True)[0][0]]=(s[np.where(condition==True)[0][0]+1]+s[np.where(condition==True)[0][0]-1])/2
s

0    NaN
1    1.0
2    1.5
3    2.0
4    NaN
5    NaN
dtype: float64

#### 【END】

### 2. 插值函数
- `interpolate`：线性插值、最近邻插值和索引插值。

对于`interpolate`而言，除了插值方法（默认为`linear`线性插值）之外，有与`fillna`类似的两个常用参数，一个是控制方向的`limit_direction`，另一个是控制最大连续缺失值插值个数的`limit`。其中，限制插值的方向默认为`forward`，这与`fillna`的`method`中的`ffill`是类似的，若想要后向限制插值或者双向限制插值可以指定为`backward`或`both`。

- 线性插值（默认）

In [81]:
s = pd.Series([np.nan, np.nan, 1, np.nan, np.nan, np.nan, 2, np.nan, np.nan])
s.values

array([nan, nan,  1., nan, nan, nan,  2., nan, nan])

In [82]:
res = s.interpolate(limit_direction='backward', limit=1)#默认方向向前
res.values

array([ nan, 1.  , 1.  ,  nan,  nan, 1.75, 2.  ,  nan,  nan])

In [85]:
res = s.interpolate(limit_direction='both', limit=1)#双向限制插值，两边各插一个
res.values

array([ nan, 1.  , 1.  , 1.25,  nan, 1.75, 2.  , 2.  ,  nan])

- 第二种常见的插值是最近邻插补，即缺失值的元素和离它最近的非缺失值元素一样：

In [21]:
s.interpolate('nearest').values

array([nan, nan,  1.,  1.,  1.,  2.,  2., nan, nan])

- 索引插值，即根据索引大小进行线性插值。例如，构造不等间距的索引进行演示：

In [86]:
s = pd.Series([0,np.nan,10],index=[0,1,10])
s

0      0.0
1      NaN
10    10.0
dtype: float64

In [88]:
s.interpolate() # 默认的线性插值，等价于计算中点的值，不看索引

0      0.0
1      5.0
10    10.0
dtype: float64

In [89]:
s.interpolate(method='index') # 和索引有关的线性插值，计算相应索引大小对应的值

0      0.0
1      1.0
10    10.0
dtype: float64

同时，这种方法对于时间戳索引也是可以使用的，有关时间序列的其他话题会在第十章进行讨论，这里举一个简单的例子：

In [90]:
s = pd.Series([0,np.nan,10], index=pd.to_datetime(['20200101', '20200102', '20200111']))
s

2020-01-01     0.0
2020-01-02     NaN
2020-01-11    10.0
dtype: float64

In [91]:
s.interpolate()

2020-01-01     0.0
2020-01-02     5.0
2020-01-11    10.0
dtype: float64

In [92]:
s.interpolate(method='index')

2020-01-01     0.0
2020-01-02     1.0
2020-01-11    10.0
dtype: float64

#### 【NOTE】关于polynomial和spline插值的注意事项
在`interpolate`中如果选用`polynomial`的插值方法，它内部调用的是`scipy.interpolate.interp1d(*,*,kind=order)`，这个函数内部调用的是`make_interp_spline`方法，因此其实是样条插值而不是类似于`numpy`中的`polyfit`多项式拟合插值；而当选用`spline`方法时，`pandas`调用的是`scipy.interpolate.UnivariateSpline`而不是普通的样条插值。这一部分的文档描述比较混乱，而且这种参数的设计也是不合理的，当使用这两类插值方法时，用户一定要小心谨慎地根据自己的实际需求选取恰当的插值方法。
#### 【END】
## 三、Nullable类型
### 1. 缺失记号及其缺陷

在`python`中的缺失值用`None`表示，该元素除了等于自己本身之外，与其他任何元素不相等：

- none

In [94]:
None == None

True

In [95]:
None == False

False

In [96]:
None == []

False

In [97]:
None == ''

False

- np.nan

在`numpy`中利用`np.nan`来表示缺失值，该元素除了不和其他任何元素相等之外，和自身的比较结果也返回`False`：

In [100]:
np.nan == np.nan#这个很有趣啊，什么都不能和np.nan相等

False

In [101]:
np.nan == None

False

In [102]:
np.nan == False

False

值得注意的是，虽然在对缺失序列或表格的元素进行比较操作的时候，`np.nan`的对应位置会返回`False`

但是在使用`equals`函数进行两张表或两个序列的相同性检验时，会自动跳过两侧表都是缺失值的位置，直接返回`True`：

In [103]:
s1 = pd.Series([1, np.nan])
s2 = pd.Series([1, 2])
s3 = pd.Series([1, np.nan])
s1 == 1

0     True
1    False
dtype: bool

In [104]:
s1.equals(s2)#s1与s2不相等

False

In [105]:
s1.equals(s3)#s1与s3用equals函数的时候是相等的

True

- pd.NaT(timedelta与datatime)

在时间序列的对象中，`pandas`利用`pd.NaT`来指代缺失值，它的作用和`np.nan`是一致的（时间序列的对象和构造将在第十章讨论）：

In [106]:
pd.to_timedelta(['30s', np.nan]) # Timedelta中的NaT

TimedeltaIndex(['0 days 00:00:30', NaT], dtype='timedelta64[ns]', freq=None)

In [107]:
pd.to_datetime(['20200101', np.nan]) # Datetime中的NaT

DatetimeIndex(['2020-01-01', 'NaT'], dtype='datetime64[ns]', freq=None)

如果出现了多个类型的元素同时存储在`Series`中，它的类型就会变成`object`。例如，同时存放整数和字符串的列表：

In [109]:
pd.Series([1, 'two'])

0      1
1    two
dtype: object

In [110]:
type(np.nan)#np.nan是一种浮点型

float

同时，由于`np.nan`的浮点性质，如果在一个整数的`Series`中出现缺失，那么其类型会转变为`float64`；而如果在一个布尔类型的序列中出现缺失，那么其类型就会转为`object`而不是`bool`：

In [111]:
pd.Series([1, np.nan]).dtype

dtype('float64')

In [112]:
pd.Series([True, False, np.nan]).dtype

dtype('O')

### 2. Nullable类型的性质

`pandas`尝试设计了一种新的缺失类型`pd.NA`以及三种`Nullable`序列类型来应对这些缺陷，它们分别是`Int, boolean`和`string`。

在上述三个`Nullable`类型中存储缺失值，都会转为`pandas`内置的`pd.NA`：

In [116]:
pd.Series([np.nan, 1], dtype = 'Int64')#int

0    <NA>
1       1
dtype: Int64

In [117]:
pd.Series([np.nan, True], dtype = 'boolean')#boolean

0    <NA>
1    True
dtype: boolean

In [118]:
pd.Series([np.nan, 'my_str'], dtype = 'string')#string

0      <NA>
1    my_str
dtype: string

对int序列加减乘除判断，都不能改变pd.Nat：

In [119]:
pd.Series([np.nan, 0], dtype = 'Int64') + 1

0    <NA>
1       1
dtype: Int64

In [121]:
pd.Series([np.nan, 0], dtype = 'Int64') == 0

0    <NA>
1    True
dtype: boolean

In [122]:
pd.Series([np.nan, 0], dtype = 'Int64') * 0.5 # 只能是浮点

0    NaN
1    0.0
dtype: float64

对于`boolean`类型的序列而言，其和`bool`序列的行为主要有两点区别：

第一点是带有缺失的布尔列表无法进行索引器中的选择，而`boolean`会把缺失值看作`False`：

In [126]:
s = pd.Series(['a', 'b'])
s_bool = pd.Series([True, np.nan])#bool序列
s_boolean = pd.Series([True, np.nan]).astype('boolean')#设定序列类型的办法一：建立的时候设定dtype，办法二：astype（）将序列转化
# s[s_bool] # 报错
s[s_boolean]

0    a
dtype: object

第二点是在进行逻辑运算时，`bool`类型在缺失处返回的永远是`False`，而`boolean`会根据逻辑运算是否能确定唯一结果来返回相应的值。那什么叫能否确定唯一结果呢？举个简单例子：`True | pd.NA`中无论缺失值为什么值，必然返回`True`；`False | pd.NA`中的结果会根据缺失值取值的不同而变化，此时返回`pd.NA`；`False & pd.NA`中无论缺失值为什么值，必然返回`False`。

In [127]:
s_boolean & True

0    True
1    <NA>
dtype: boolean

In [128]:
s_boolean | True

0    True
1    True
dtype: boolean

In [129]:
s_bool | True

0     True
1    False
dtype: bool

In [133]:
~s_boolean # 取反操作同样是无法唯一地判断缺失结果

0    False
1     <NA>
dtype: boolean

关于`string`类型的具体性质将在下一章文本数据中进行讨论。

一般在实际数据处理时，可以在数据集读入后，先通过`convert_dtypes`转为`Nullable`类型：

In [135]:
df = pd.read_csv('C:\datawhale\joyful-pandas-master\data\learn_pandas.csv')
df = df.convert_dtypes()#convert_dtypes转化成nullable类型
df.dtypes

School          string
Grade           string
Name            string
Gender          string
Height         float64
Weight           Int64
Transfer        string
Test_Number      Int64
Test_Date       string
Time_Record     string
dtype: object

### 3. 缺失数据的计算和分组

- 当调用函数`sum, prob`使用加法和乘法的时候，缺失数据等价于被分别视作0和1，即不改变原来的计算结果：

In [136]:
s = pd.Series([2,3,np.nan,4,5])
s.sum()

14.0

In [137]:
s.prod()

120.0

- 当使用cumsum时，会自动跳过缺失值所处的位置：

In [138]:
s.cumsum()

0     2.0
1     5.0
2     NaN
3     9.0
4    14.0
dtype: float64

- 运算

In [139]:
np.nan == 0#np.nan六亲不认

False

In [142]:
pd.NA == 0

<NA>

In [143]:
np.nan > 0

False

In [144]:
pd.NA > 0

<NA>

In [145]:
np.nan + 1

nan

In [146]:
np.log(np.nan)

nan

In [147]:
np.add(np.nan, 1)

nan

In [148]:
np.nan ** 0

1.0

In [149]:
pd.NA ** 0

1

In [150]:
1 ** np.nan

1.0

In [151]:
1 ** pd.NA

1

另外需要注意的是，`diff, pct_change`这两个函数虽然功能相似，但是对于缺失的处理不同，前者凡是参与缺失计算的部分全部设为了缺失值，而后者缺失值位置会被设为 0% 的变化率：

In [152]:
s.diff()#后一个减前一个

0    NaN
1    1.0
2    NaN
3    NaN
4    1.0
dtype: float64

In [153]:
s.pct_change()

0         NaN
1    0.500000
2    0.000000
3    0.333333
4    0.250000
dtype: float64

缺失可以作为一个类别处理，例如在`groupby, get_dummies`中可以设置相应的参数来进行增加缺失类别：

In [154]:
df_nan = pd.DataFrame({'category':['a','a','b',np.nan,np.nan], 'value':[1,3,5,7,9]})
df_nan

category  value
0        a      1
1        a      3
2        b      5
3      NaN      7
4      NaN      9

In [156]:
df_nan.groupby('category', dropna=False)['value'].mean() #默认忽视nan值，将dropna设置成false就可以新增nan类

category
a      2
b      5
NaN    8
Name: value, dtype: int64

In [158]:
pd.get_dummies(df_nan.category, dummy_na=True)#有时为1，无时为0

a  b  NaN
0  1  0    0
1  1  0    0
2  0  1    0
3  0  0    1
4  0  0    1

## 四、练习
### Ex1：缺失值与类别的相关性检验
在数据处理中，含有过多缺失值的列往往会被删除，除非缺失情况与标签强相关。下面有一份关于二分类问题的数据集，其中`X_1, X_2`为特征变量，`y`为二分类标签。

In [159]:
df = pd.read_csv('C:\datawhale\joyful-pandas-master\data\missing_chi.csv')
df.head()

X_1  X_2  y
0   NaN  NaN  0
1   NaN  NaN  0
2   NaN  NaN  0
3  43.0  NaN  0
4   NaN  NaN  0

In [168]:
df.isna().mean()#查看每列isna的比例

X_1    0.855
X_2    0.894
y      0.000
dtype: float64

In [169]:
df.y.value_counts(normalize=True)#标准化看y中有多少个0，多少个1

0    0.918
1    0.082
Name: y, dtype: float64

事实上，有时缺失值出现或者不出现本身就是一种特征，并且在一些场合下可能与标签的正负是相关的。关于缺失出现与否和标签的正负性，在统计学中可以利用卡方检验来断言它们是否存在相关性。按照特征缺失的正例、特征缺失的负例、特征不缺失的正例、特征不缺失的负例，可以分为四种情况，设它们分别对应的样例数为$n_{11}, n_{10}, n_{01}, n_{00}$。假若它们是不相关的，那么特征缺失中正例的理论值，就应该接近于特征缺失总数$\times$总体正例的比例，即：

$$E_{11} = n_{11} \approx (n_{11}+n_{10})\times\frac{n_{11}+n_{01}}{n_{11}+n_{10}+n_{01}+n_{00}} = F_{11}$$

其他的三种情况同理。现将实际值和理论值分别记作$E_{ij}, F_{ij}$，那么希望下面的统计量越小越好，即代表实际值接近不相关情况的理论值：

$$S = \sum_{i\in \{0,1\}}\sum_{j\in \{0,1\}} \frac{(E_{ij}-F_{ij})^2}{F_{ij}}$$

可以证明上面的统计量近似服从自由度为$1$的卡方分布，即$S\overset{\cdot}{\sim} \chi^2(1)$。因此，可通过计算$P(\chi^2(1)>S)$的概率来进行相关性的判别，一般认为当此概率小于$0.05$时缺失情况与标签正负存在相关关系，即不相关条件下的理论值与实际值相差较大。

上面所说的概率即为统计学上关于$2\times2$列联表检验问题的$p$值， 它可以通过`scipy.stats.chi2(S, 1)`得到。请根据上面的材料，分别对`X_1, X_2`列进行检验。


- 方法一

In [186]:
cat_1 = df.X_1.fillna('NaN').mask(df.X_1.notna()).fillna("NotNaN")#将x1，x2变成nan与notnan的序列
cat_2 = df.X_2.fillna('NaN').mask(df.X_2.notna()).fillna("NotNaN")

In [179]:
pd.crosstab(cat_1, df.y)#交叉表

y         0   1
X_1            
NaN     785  70
NotNaN  133  12

In [188]:
df_1 = pd.crosstab(cat_1, df.y, margins=True)
df_1

y         0   1   All
X_1                  
NaN     785  70   855
NotNaN  133  12   145
All     918  82  1000

In [189]:
df_2 = pd.crosstab(cat_2, df.y, margins=True)
df_2

y         0   1   All
X_2                  
NaN     894   0   894
NotNaN   24  82   106
All     918  82  1000

In [193]:
def compute_S(my_df):
    S = []
    for i in range(2):
        for j in range(2):
            E = my_df.iloc[i, j]
            F = my_df.iat[i, 2]*my_df.iat[2, j]/my_df.iat[2,2]
            S.append((E-F)**2/F)
    return sum(S)

res1 = compute_S(df_1)
res1

0.0012965662713972017

In [194]:
res2 = compute_S(df_2)
res2

753.3604636823281

In [201]:
from scipy.stats import chi2
chi2.sf(res1, 1)# X_1检验的p值大于0.05 不能认为相关，剔除

0.9712760884395901

In [202]:
chi2.sf(res2, 1)# X_2检验的p值小于0.05 认为相关，保留

7.459641265637543e-166

- 方法二

In [184]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(cat_1, df.y), correction=False)[1]#不相关条件

0.9712760884395901

In [185]:
chi2_contingency(pd.crosstab(cat_2, df.y), correction=False)[1]

7.459641265637543e-166

### Ex2：用回归模型解决分类问题

`KNN`是一种监督式学习模型，既可以解决回归问题，又可以解决分类问题。对于分类变量，利用`KNN`分类模型可以实现其缺失值的插补，思路是度量缺失样本的特征与所有其他样本特征的距离，当给定了模型参数`n_neighbors=n`时，计算离该样本距离最近的$n$个样本点中最多的那个类别，并把这个类别作为该样本的缺失预测类别

上面有色点的特征数据提供如下：

In [203]:
df = pd.read_excel('C:\datawhale\joyful-pandas-master\data\color.xlsx')
df.head(3)

X1   X2 Color
0 -2.5  2.8  Blue
1 -1.5  1.8  Blue
2 -0.8  2.8  Blue

已知待预测的样本点为$X_1=0.8, X_2=-0.2$，那么预测类别可以如下写出：

In [204]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=6)
clf.fit(df.iloc[:,:2], df.Color)
clf.predict([[0.8, -0.2]])

array(['Yellow'], dtype=object)

1. 对于回归问题而言，需要得到的是一个具体的数值，因此预测值由最近的$n$个样本对应的平均值获得。请把上面的这个分类问题转化为回归问题，仅使用`KNeighborsRegressor`来完成上述的`KNeighborsClassifier`功能。

In [247]:
df = pd.read_excel('C:\\datawhale\\joyful-pandas-master\\data\\color.xlsx')
df.head(3)

X1   X2 Color
0 -2.5  2.8  Blue
1 -1.5  1.8  Blue
2 -0.8  2.8  Blue

In [248]:
from sklearn.neighbors import KNeighborsRegressor
df_dummies = pd.get_dummies(df.Color)#先转化成01类的标记
stack_list = []
for col in df_dummies.columns:
    clf = KNeighborsRegressor(n_neighbors=6)
    clf.fit(df.iloc[:,:2], df_dummies[col])
    res = clf.predict([[0.8, -0.2]])
    stack_list.append(res)
    
code_res = pd.Series(np.hstack(stack_list).argmax())#将stack水平拼接后寻找最大的数值的位置索引
df_dummies.columns[code_res[0]]

'Yellow'

2. 请根据第1问中的方法，对`audit`数据集中的`Employment`变量进行缺失值插补。

In [304]:
from sklearn.neighbors import KNeighborsRegressor
df = pd.read_csv('C:\\datawhale\\joyful-pandas-master\\data\\audit.csv')
res_df = df.copy()
res_df.head()

ID  Age Employment    Marital     Income  Gender  Hours
0  1004641   38    Private  Unmarried   81838.00  Female     72
1  1010229   35    Private     Absent   72099.00    Male     30
2  1024587   32    Private   Divorced  154676.74    Male     40
3  1038288   45    Private    Married   27743.82    Male     55
4  1044221   60    Private    Married    7568.23    Male     40

In [287]:
df1 = pd.concat([pd.get_dummies(df[['Marital', 'Gender']]),df[['Age','Income','Hours']].apply(lambda x:(x-x.min())/(x.max()-x.min())), df.Employment],1)
#将婚姻状况和性别转化成01，将年纪，收入与小时标准化，将雇佣作为需要预测的，这三者拼接到一起
df1.head(3)

Marital_Absent  Marital_Divorced  Marital_Married  \
0               0                 0                0   
1               1                 0                0   
2               0                 1                0   

   Marital_Married-spouse-absent  Marital_Unmarried  Marital_Widowed  \
0                              0                  1                0   
1                              0                  0                0   
2                              0                  0                0   

   Gender_Female  Gender_Male       Age    Income     Hours Employment  
0              1            0  0.287671  0.168997  0.724490    Private  
1              0            1  0.246575  0.148735  0.295918    Private  
2              0            1  0.205479  0.320539  0.397959    Private

In [301]:
X_train = df1[df1.Employment.notna()]
X_test = df1[df1.Employment.isna()]#分为训练集和测试集

In [305]:
df_dummies = pd.get_dummies(X_train.Employment)
stack_list = []
for col in df_dummies.columns:
    clf = KNeighborsRegressor(n_neighbors=6)
    clf.fit(X_train.iloc[:,:-1], df_dummies[col])#将训练集除了最后一列的所有数据与最后一列的标签做训练
    res = clf.predict(X_test.iloc[:,:-1]).reshape(-1,1)#预测测试机的所有
    stack_list.append(res)
    
code_res = pd.Series(np.hstack(stack_list).argmax(1))#将stack水平拼接后寻找最大的数值的位置索引
cat_res = code_res.replace(dict(zip(list(range(df_dummies.shape[0])),df_dummies.columns)))#将缺测值填充好
res_df.loc[res_df.Employment.isna(), 'Employment'] = cat_res.values#放回去原列表，改变雇佣列

In [306]:
res_df.head()

ID  Age Employment    Marital     Income  Gender  Hours
0  1004641   38    Private  Unmarried   81838.00  Female     72
1  1010229   35    Private     Absent   72099.00    Male     30
2  1024587   32    Private   Divorced  154676.74    Male     40
3  1038288   45    Private    Married   27743.82    Male     55
4  1044221   60    Private    Married    7568.23    Male     40

In [308]:
res_df.isna().sum()#没有缺测值了

ID            0
Age           0
Employment    0
Marital       0
Income        0
Gender        0
Hours         0
dtype: int64